
# Structured Outputs (Retail Theme)

**What you'll explore**
- How to guide models to return valid, structured JSON instead of free-form text
- Enforcing schemas with Pydantic models for reliability and validation
- Practical examples across domains:

    - calendar event creation

    - chain-of-thought math tutoring

    - pros/cons from reviews

    - product catalog extraction

    - POS / app log parsing
    
    - Receipt Image to Structure JSON 

**Exercises**

At the end of the notebook, you’ll find 5 exercises to practice these skills.

In [17]:

from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import List, Literal, Optional
import os,json, textwrap, base64

# Optional: set your key here for local testing (avoid committing real keys)
os.environ["OPENAI_API_KEY"] = "" 

client = OpenAI()

MODEL = "gpt-4o-mini"


## 1) Calendar Event Extraction

Turn free-form text about plans into a structured calendar event with name, date, and participants.”

In [ ]:
# Define a structured schema for events using Pydantic.
# This enforces that every event has:
# - event_name (string)
# - date (string, e.g. "Saturday" or "2025-08-31")
# - participants (list of strings)
class CalendarEvent(BaseModel):
    event_name: str
    date: str
    participants: list[str]


# Ask the model to extract event info from a natural language sentence.
# Because we pass response_format=CalendarEvent, the model must return JSON
# that matches the schema, and Pydantic will validate it.
response = client.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Bruce and Arthur are going to a boxing match on Saturday.",
        },
    ],
    response_format=CalendarEvent,
)

# The parsed, schema-validated object is accessible here.
event = response.choices[0].message.parsed

# Print the structured CalendarEvent object
print(event)


event_name='Boxing Match' date='Next Weekend' participants=['Bruce', 'Arthur']


## 2) Structured Outputs for Chain-of-Thought Math Tutoring
Capture step-by-step reasoning and a final answer in a structured format instead of free text.

In [26]:
# Define a schema for one step in the solution
class Step(BaseModel):
    explanation: str   # plain-language reasoning for this step
    output: str        # the numeric or algebraic result of this step

# Define a schema for the full reasoning process
class MathReasoning(BaseModel):
    steps: list[Step]   # ordered list of reasoning steps
    final_answer: str   # final solution at the end

# Ask the model to act like a math tutor and guide through the solution
response = client.chat.completions.parse(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful math tutor. Guide the user through the solution step by step.",
        },
        {"role": "user", "content": "how can I solve 8x + 7 = -23"},
    ],
    response_format=MathReasoning,
)

# The structured reasoning object returned by the model
math_reasoning = response.choices[0].message.parsed
print(math_reasoning)


steps=[Step(explanation='We start with the equation 8x + 7 = -23. The first step is to isolate the term with x by subtracting 7 from both sides.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='This simplifies to 8x = -30. Now we need to solve for x by dividing both sides by 8.', output='8x / 8 = -30 / 8'), Step(explanation='This simplifies to x = -30 / 8, which can be reduced to x = -15 / 4.', output='x = -15/4')] final_answer='x = -3.75'


## 3) Reviews → Pros/Cons
Turn free-form customer feedback into a structured list of pros and cons.

In [37]:
# Define a schema for structured sentiment extraction
class ProsCons(BaseModel):
    pros: List[str]   # positive aspects
    cons: List[str]   # negative aspects

# Example customer review (unstructured text)
review = "Delivery was late, but the shoes fit well and are very comfortable."
# Ask the model to extract pros and cons into the schema
resp = client.chat.completions.parse(
    model=MODEL,
    messages=[{
        "role": "user",
        "content": f"Extract the pros and cons from the review.\n\nReview: {review}"
    }],
    temperature=0,              
    response_format=ProsCons
)

# The structured object with pros/cons lists
data = resp.choices[0].message.parsed
print(data)


pros=['Shoes fit well', 'Very comfortable'] cons=['Delivery was late']


## Catalog Extraction (brand/model/price…)

Extracting structured product data from unstructured description

In [ ]:
# Define a schema for catalog entries
class CatalogEntry(BaseModel):
    brand: str
    model: str
    storage: Optional[str] = None   # optional if not mentioned
    color: Optional[str] = None     # optional if not mentioned
    price: float

# Example product description (unstructured text)
desc = """Apple iPhone 14 Pro Max – 256GB, Space Black

Experience power, elegance, and cutting-edge technology with the iPhone 14 Pro Max. Finished in a sleek Space Black, this flagship device combines premium design with uncompromising performance.

256GB storage ensures plenty of space for photos, apps, and 4K videos.

Powered by Apple’s advanced A16 Bionic chip, it delivers lightning-fast speed and smooth multitasking.

The Super Retina XDR display with ProMotion brings vibrant colors and ultra-fluid visuals.

Capture stunning detail in any light with the 48MP triple-camera system and cinematic video modes.

Built with iOS features like Dynamic Island, Always-On display, and enhanced security.

Whether for work, creativity, or play, the iPhone 14 Pro Max is designed to keep up with your life — all for $1299."""

# Ask the model to extract structured catalog fields
resp2 = client.chat.completions.parse(
    model=MODEL,
    messages=[{"role": "user", "content": f"Extract catalog fields from:\n{desc}."}],
    temperature=0,
    response_format=CatalogEntry
)

# The structured object with product attributes
entry = resp2.choices[0].message.parsed
print(entry)


brand='Apple' model='iPhone 14 Pro Max' storage='256GB' color='Space Black' price=0.0


## POS / App Log Parsing
Convert raw log lines into structured records with severity, error codes, and metadata OR flag as refusal if unparseable.

In [30]:
# Define a schema for structured log records
class LogRecord(BaseModel):
    timestamp: str
    severity: Literal["low","medium","high","critical"]   # mapped from INFO/WARN/ERROR/FATAL
    error_code: str
    endpoint: Optional[str] = None
    user_id: Optional[str] = None
    register_id: Optional[str] = None
    message: str
    tags: List[str] = []     # extracted keywords
    status: Literal["success","refusal"]  # "refusal" if parsing fails

# Example log lines
log_line  = """[2025-03-01 14:22:33] ERROR 502: Gateway timeout on /orders/checkout user=abc123 register=12"""
log_line1 = """2025-03-01T07:10:05Z WARN 429 Too many requests"""
log_line2 = """<<<garbled>>> ### %% no time no code /???"""

# Instruction prompt to the model
instruction = f"""Parse the log into JSON. 
If unparsable/unsafe, set status="refusal" and keep only message with reason. 
Return JSON only.

Log: {log_line}
"""

# Ask the model to parse the log into a LogRecord schema
resp3 = client.chat.completions.parse(
    model=MODEL,
    messages=[{"role":"user","content": instruction}],
    temperature=0,
    response_format=LogRecord
)

# Structured object with parsed log record (or refusal)
record = resp3.choices[0].message.parsed
print(record)


timestamp='2025-03-01 14:22:33' severity='high' error_code='502' endpoint='/orders/checkout' user_id='abc123' register_id='12' message='Gateway timeout' tags=[] status='success'


## Receipt → Structured JSON 
Use a vision call to read a receipt image, then convert the extracted text into a validated JSON schema.

In [32]:
from typing import List
from pydantic import BaseModel
from IPython.display import Image, display
import base64

# -------- First pass — Vision: extract receipt text from a local image --------
local_image_path = "receipt_1.jpeg"  # <- set to a real file on your machine
display(Image(url=local_image_path))

try:
    with open(local_image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    data_url = f"data:image/jpeg;base64,{b64}"

    resp2 = client.chat.completions.create(
        model=MODEL,  # vision-capable model
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": "Extract store name, city/state if visible, date, items ordered and total amount."},
                {"type": "image_url", "image_url": {"url": data_url}}
            ]
        }],
        temperature=0.2
    )
    print(resp2.choices[0].message.content)  # raw extracted text

except FileNotFoundError:
    print("⚠️ Set local_image_path to a real image before running.")

# -------- Second pass — Structured: convert text → validated JSON (Receipt) --------
class ReceiptItem(BaseModel):
    name: str
    quantity: float
    unit_price: float
    line_total: float

class Receipt(BaseModel):
    store_name: str
    store_city: str
    store_state: str
    date: str
    items: List[ReceiptItem]
    total: float

receipt_text = resp2.choices[0].message.content
prompt = f"Extract a structured receipt from this text.\n\n{receipt_text}"

resp4 = client.chat.completions.parse(
    model=MODEL,  # text model
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
    response_format=Receipt
)

receipt = resp4.choices[0].message.parsed
print(receipt)


- **Store Name:** The Tack Room
- **City/State:** Lincoln, MA
- **Date:** 4/8/24
- **Items Ordered:**
  1. BBQ Potato Chips - $7.00
  2. Diet Coke - $1.00
  3. Trillium Fort Point - $6.00
  4. Fried Chicken Sandwich - $14.00
  5. Famous Duck Grilled Cheese - $12.00
  6. Mac & Cheese - $6.00
  7. Burger of the moment - $14.00
- **Total Amount:** $124.53
store_name='The Tack Room' store_city='Lincoln' store_state='MA' date='4/8/24' items=[ReceiptItem(name='BBQ Potato Chips', quantity=1.0, unit_price=7.0, line_total=7.0), ReceiptItem(name='Diet Coke', quantity=1.0, unit_price=1.0, line_total=1.0), ReceiptItem(name='Trillium Fort Point', quantity=1.0, unit_price=6.0, line_total=6.0), ReceiptItem(name='Fried Chicken Sandwich', quantity=1.0, unit_price=14.0, line_total=14.0), ReceiptItem(name='Famous Duck Grilled Cheese', quantity=1.0, unit_price=12.0, line_total=12.0), ReceiptItem(name='Mac & Cheese', quantity=1.0, unit_price=6.0, line_total=6.0), ReceiptItem(name='Burger of the moment', qu

# 🧾 Exercises — Structured Outputs

### Exercise 1: Pros/Cons with Sarcasm  
Take a sarcastic product review like:  
*"Awesome, the phone died in two days. At least it’s a paperweight now."*  
- Extract pros and cons as lists.  
- Check if the model correctly treats sarcasm as negative instead of positive.  

---

### Exercise 2: Catalog Parsing with Missing Info  
You’re given a product description without a price, e.g.:  
*"Samsung Galaxy S22 Ultra – 256GB, Phantom Black. A premium smartphone with pro-grade cameras and S Pen."*  
- Extract catalog fields: brand, model, storage, color, and price.  
- Observe whether the model invents a price, leaves it empty, or defaults to zero.  

---

### Exercise 3: Log Parsing with Mixed Severities  
You’re given logs with different severities: INFO, WARN, ERROR, CRITICAL. 

2025-03-01T07:10:05Z INFO User 123 logged in

2025-03-01T07:12:44Z WARN Disk space at 85% on server-7

2025-03-01T07:15:10Z ERROR 502 Gateway timeout on /orders/checkout

2025-03-01T07:16:22Z CRITICAL Database connection lost on cluster-2

2025-03-01T07:18:01Z DEBUG Cache refreshed for /products/12345

- Parse the logs into structured objects. 
- Improve the prompt by explicitly including a severity mapping:  
  - INFO/DEBUG → low  
  - WARN → medium  
  - ERROR → high  
  - CRITICAL → critical  

---

### Exercise 4: GST Invoice To Structured Output
You’re given a GST invoice image — https://www.outputbooks.com/wp-content/themes/outputbooks/images/oub_GST_Invoice_Format.png  
- **Pass 1 (Vision):** Extract all visible text from the invoice.  
- **Pass 2 (Structured):** From the extracted text, structure the data into:  
  - invoice number, invoice date, due date  
  - seller details (company, address, GSTIN)  
  - buyer details (name, address, GSTIN, contact)  
  - line items (name, manufacturing date, quantity, rate, tax, amount)  
  - subtotal
  - grand total  
  - amount in words  

---

### Exercise 5: Design Your Own Structured Scenario  
Think of a business case where structured JSON would be more useful than free text.  
Write a short schema, craft a prompt, and run it on your own input.

